# Datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("princeton-nlp/llama3-ultrafeedback-armorm")
ds.save_to_disk("/home/hanyang/RainbowPO/examples/raw_datasets/llama3-ultrafeedback-armorm")

In [ ]:
# To trl format
from datasets import load_from_disk
ds = load_from_disk(f"/home/hanyang/RainbowPO/examples/raw_datasets/llama3-ultrafeedback-armorm-trl")
ds_rlhf = ds
ds_rlhf = ds_rlhf.remove_columns(['all_generated_responses', 'all_rm_scores'])
ds_rlhf.save_to_disk(f"/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_trl/")

In [ ]:
# Check Dataset
from datasets import load_from_disk
dataset_path = "/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_rso_trl"
UltraFeedback_RM_trl= load_from_disk(dataset_path)
UltraFeedback_RM_trl["train"]

# RSO

In [ ]:
from datasets import load_dataset,DatasetDict

# Load the dataset from a JSON file
dataset = load_dataset('json', data_files='/home/hanyang/SimPO/on_policy_data_gen/datasets/llama3_ultrafeedback/processed_results.json')

# Get the total length of the dataset
dataset_length = len(dataset['train'])

# Define the number of examples in the test set
test_size = 1961

# Split the dataset
train_dataset = dataset['train'].select(range(dataset_length - test_size))  # All but last 1961 prompts
test_dataset = dataset['train'].select(range(dataset_length - test_size, dataset_length))  # Last 1961 prompts

# Create a DatasetDict with 'train' and 'test' splits
split_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# split_dataset.save_to_disk('examples/raw_datasets/llama3-ultrafeedback-armorm-rso')

In [ ]:
dataset = load_dataset('json', data_files='/home/hanyang/SimPO/on_policy_data_gen/datasets/llama3_ultrafeedback/all_outputs.json')

In [ ]:
dataset

In [ ]:
from datasets import load_dataset,DatasetDict

for alpha in [0.2, 0.5, 1.0, 2.0, 5.0]:

    # Load the dataset from a JSON file
    dataset = load_dataset('json', data_files=f'/home/hanyang/SimPO/on_policy_data_gen/datasets/llama3_ultrafeedback/alpha={alpha}.json')

    dataset.save_to_disk(f'examples/raw_datasets/llama3-ultrafeedback-armorm-rso-alpha={alpha}')

## Process Dataset to trl format

In [ ]:
from datasets import load_from_disk
for alpha in [0.2, 0.5, 1.0, 2.0, 5.0]:
    ds = load_from_disk(f"/home/hanyang/RainbowPO/examples/raw_datasets/llama3-ultrafeedback-armorm-rso-alpha={alpha}")
    ds_rlhf = ds
    ds_rlhf = ds_rlhf.remove_columns(['all_generated_responses', 'all_rm_scores'])
    ds_rlhf.save_to_disk(f"/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_rso_alpha={alpha}_trl/")

## DataSet Checking

In [13]:
from datasets import load_from_disk
alpha = 0.2
dataset_path = f"/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_rso_alpha={alpha}_trl"
UltraFeedback_RM_trl= load_from_disk(dataset_path)
UltraFeedback_RM_trl["train"]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 60077
})

# KTO

In [1]:
from datasets import load_from_disk
from tqdm import tqdm

dpo_dataset_dict = load_from_disk("/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_trl")

kto_dataset_dict = load_from_disk("/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_kto_trl")


In [ ]:
from datasets import load_dataset
from datasets import load_from_disk
from tqdm import tqdm

dpo_dataset_dict = load_dataset("<some path>/RainbowPO/raw_datasets/llama3-ultrafeedback-armorm/")

def convert_dataset(dpo_subset):
    n = len(dpo_subset['prompt'])
   
    # Pre-allocate lists for efficiency
    kto_prompt = [None] * (2 * n)  # each prompt will have two entries (chosen and rejected)
    kto_completion = [None] * (2 * n)
    kto_label = [None] * (2 * n)
   
    # Use tqdm for progress bar
    for i, row in tqdm(enumerate(dpo_subset), desc="Converting dataset"):
        # Populate the lists
        kto_prompt[2 * i] = row["prompt"]
        kto_prompt[2 * i + 1] = row["prompt"]

        # First, add the chosen response with a True label
        kto_completion[2 * i] = row["chosen"]
        kto_label[2 * i] = True

        # Then, add the rejected response with a False label
        kto_completion[2 * i + 1] = row["rejected"]
        kto_label[2 * i + 1] = False

    return {
        "prompt": kto_prompt,
        "completion": kto_completion,
        "label": kto_label
    }

# Convert both train and test sets
kto_dataset_dict = {
    "train": convert_dataset(dpo_dataset_dict["train"]),
    "test": convert_dataset(dpo_dataset_dict["test"])
}

# Save the converted dataset
kto_dataset_dict.save_to_disk("/home/hanyang/RainbowPO/examples/datasets/UltraFeedback_armorm_kto_trl")

# RainbowPO

In [7]:
dataset = load_from_disk('examples/raw_datasets/llama3-ultrafeedback-armorm-rso-alpha=0.2')

In [10]:
dataset["train"][0]

{'prompt': '\n\nA federal judge on Thursday rejected efforts by Texas to stop the resettlement of Syrian refugees within its borders. The ruling comes days after presumptive Republican presidential nominee Donald Trump called to block non-citizen Muslims from entering the U.S. following the Orlando nightclub massacre. Even before the attack, Americans were divided, mostly along party lines, on the issue of refugees from war-torn countries in the Middle East. A survey conducted in late May by the Brookings Institution found of those polled, just 38 percent of Republicans supported taking in refugees from Syria and the Middle East, compared with 77 percent of Democrats. But among Trump supporters, an overwhelming 77 percent said they oppose taking in refugees. So who are these people who cause such a divide among Americans, triggering feelings of dread and suspicion in some and feelings of empathy and hospitality in others? Millions have been forced to flee the bloody conflicts in Syria,